In [ ]:
from pyexpat.errors import messages
!poetry remove wikipedia
!poetry add wikipedia-api

In [ ]:
!poetry add numpy

In [ ]:
from ai_agent_experiments.config import Configuration

config=Configuration("./config.json")
messages=[{"role":"system", "content":"You are a Helpful person who has deep interest in speaking history in the form of anecdotes."}]

In [ ]:
from openai import AzureOpenAI, embeddings

client= AzureOpenAI(api_key= config.client_config["api_key"], azure_endpoint=config.client_config["azure_endpoint"],api_version=config.client_config["api_version"] )
response=client.chat.completions.create(model=config.client_config["model"],messages=messages)
print(response.choices[0].message.content)

In [ ]:
from wikipediaapi import Wikipedia
wiki = Wikipedia("RAGBot/0.0", 'en')
doc_page = wiki.page("Python_(programming_language)").text
chunks = doc_page.split("\n\n") # Splitting by double newlines for paragraphs



In [ ]:
embeddings=client.embeddings.create(input=chunks, model="text-embedding-ada-002")

In [ ]:
print(embeddings.data[0].embedding)

In [ ]:
user_query="tell me the history of python programming language"
user_query_embedding=client.embeddings.create(input=[user_query], model="text-embedding-ada-002")

In [ ]:
import numpy as np

def find_most_similar_chunks_numpy(query_vector, chunk_vectors, chunks, top_k=3):
    """
    Pure NumPy implementation - no external dependencies
    """
    query = np.array(query_vector)
    chunk_arrays = np.array(chunk_vectors)

    # Compute cosine similarities
    similarities = np.dot(chunk_arrays, query) / (
        np.linalg.norm(chunk_arrays, axis=1) * np.linalg.norm(query)
    )

    # Get top-k indices
    top_indices = np.argsort(similarities)[::-1][:top_k]

    results = []
    for idx in top_indices:
        results.append({
            'chunk': chunks[idx],
            'similarity': similarities[idx],
            'index': idx
        })

    return results


In [ ]:
# Extract embeddings from the response objects
chunk_vectors = [item.embedding for item in embeddings.data]
query_vector = user_query_embedding.data[0].embedding

top_chunks = find_most_similar_chunks_numpy(query_vector, chunk_vectors, chunks, top_k=3)

In [ ]:
top_chunks

In [ ]:
def generate_rag_response(openai_client, user_input, retrieved_chunks):
    content_chunks = "\n\n".join([chunk['chunk'] for chunk in retrieved_chunks])
    updated_content = f"""Based on the following context, answer the user's question. If the answer cannot be found in the context, say so.

    Context:{content_chunks}
    Question: {user_query}
    """
    messages.append({"role": "user", "content": updated_content})
    response = openai_client.chat.completions.create(
        model=config.client_config["model"],
        messages=messages)
    return response.choices[0].message.content

In [ ]:
print(generate_rag_response(client, user_query, top_chunks))